discuss if false negative or false positive is worse. here its worse to lose a customer when you think you have them, then to think ull lose one when u rly wont. So false negative, bc here if churn is true it means u lost the customer, so false negative means u predict no churn when they really do

In [105]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import precision_score, recall_score
from sklearn.model_selection import train_test_split, GridSearchCV,\
cross_val_score, RandomizedSearchCV

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [106]:
!ls data

archive.zip                        bigml_59c28831336c6604c800002a.csv


In [107]:
df = pd.read_csv('data/bigml_59c28831336c6604c800002a.csv')
df.head()

,state,account length,area code,phone number,international plan,voice mail plan,number vmail messages,total day minutes,total day calls,total day charge,...,total eve calls,total eve charge,total night minutes,total night calls,total night charge,total intl minutes,total intl calls,total intl charge,customer service calls,churn
0,KS,128,415,382-4657,no,yes,25,265.1,110,45.07,...,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,371-7191,no,yes,26,161.6,123,27.47,...,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,358-1921,no,no,0,243.4,114,41.38,...,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,375-9999,yes,no,0,299.4,71,50.90,...,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,330-6626,yes,no,0,166.7,113,28.34,...,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


In [108]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3333 entries, 0 to 3332
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   state                   3333 non-null   object 
 1   account length          3333 non-null   int64  
 2   area code               3333 non-null   int64  
 3   phone number            3333 non-null   object 
 4   international plan      3333 non-null   object 
 5   voice mail plan         3333 non-null   object 
 6   number vmail messages   3333 non-null   int64  
 7   total day minutes       3333 non-null   float64
 8   total day calls         3333 non-null   int64  
 9   total day charge        3333 non-null   float64
 10  total eve minutes       3333 non-null   float64
 11  total eve calls         3333 non-null   int64  
 12  total eve charge        3333 non-null   float64
 13  total night minutes     3333 non-null   float64
 14  total night calls       3333 non-null   

In [109]:
df['area code'] = df['area code'].astype(object)

df.dtypes

state                      object
account length              int64
area code                  object
phone number               object
international plan         object
voice mail plan            object
number vmail messages       int64
total day minutes         float64
total day calls             int64
total day charge          float64
total eve minutes         float64
total eve calls             int64
total eve charge          float64
total night minutes       float64
total night calls           int64
total night charge        float64
total intl minutes        float64
total intl calls            int64
total intl charge         float64
customer service calls      int64
churn                        bool
dtype: object

In [110]:
y = df.churn
X = df.drop(['churn', 'phone number'], axis = 1)

In [111]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)

In [112]:
X_train_cat = X_train.select_dtypes('object')

ohe = OneHotEncoder(drop='first', sparse=False)

dums = ohe.fit_transform(X_train_cat)
dums_df = pd.DataFrame(dums, columns=ohe.get_feature_names(), index=X_train_cat.index)

In [113]:
dums_df.head()

,x0_AL,x0_AR,x0_AZ,x0_CA,x0_CO,x0_CT,x0_DC,x0_DE,x0_FL,x0_GA,...,x0_VA,x0_VT,x0_WA,x0_WI,x0_WV,x0_WY,x1_415,x1_510,x2_yes,x3_yes
817,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1373,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
679,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
56,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1993,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [114]:
X_train_nums = X_train.select_dtypes(['int64', 'float64'])

ss = StandardScaler()

ss.fit(X_train_nums)

nums_df = pd.DataFrame(ss.transform(X_train_nums), columns=X_train_nums.columns, index=X_train_nums.index)

In [115]:
nums_df.head()

,account length,number vmail messages,total day minutes,total day calls,total day charge,total eve minutes,total eve calls,total eve charge,total night minutes,total night calls,total night charge,total intl minutes,total intl calls,total intl charge,customer service calls
817,3.601382,-0.584936,-1.547653,-0.429657,-1.547170,-0.729987,-1.840891,-0.731087,1.255804,0.925634,1.256197,-1.300791,0.634849,-1.304132,0.318978
1373,0.184951,-0.584936,-1.244014,0.224176,-1.244071,-0.138082,0.499864,-0.139179,0.165090,-0.353704,0.164841,-2.194793,-0.184370,-2.191525,1.813519
679,-0.650176,-0.584936,0.787609,-1.133785,0.787772,2.491952,0.549667,2.493068,0.147339,0.209205,0.147309,-0.549828,1.863677,-0.549186,-0.428293
56,1.020079,-0.584936,-0.969818,-0.127888,-0.970200,-0.408385,-1.890695,-0.408439,-1.178086,1.437368,-1.176344,-0.800149,-1.003589,-0.800835,-0.428293
1993,-0.371801,-0.584936,0.675354,-0.228477,0.675192,1.294330,-1.143645,1.295326,0.265680,0.516246,0.265649,-2.051753,-0.593980,-2.045833,-1.175564


In [116]:
X_train_clean = pd.concat([nums_df, dums_df], axis=1)

X_train_clean.head()

,account length,number vmail messages,total day minutes,total day calls,total day charge,total eve minutes,total eve calls,total eve charge,total night minutes,total night calls,...,x0_VA,x0_VT,x0_WA,x0_WI,x0_WV,x0_WY,x1_415,x1_510,x2_yes,x3_yes
817,3.601382,-0.584936,-1.547653,-0.429657,-1.547170,-0.729987,-1.840891,-0.731087,1.255804,0.925634,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1373,0.184951,-0.584936,-1.244014,0.224176,-1.244071,-0.138082,0.499864,-0.139179,0.165090,-0.353704,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
679,-0.650176,-0.584936,0.787609,-1.133785,0.787772,2.491952,0.549667,2.493068,0.147339,0.209205,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
56,1.020079,-0.584936,-0.969818,-0.127888,-0.970200,-0.408385,-1.890695,-0.408439,-1.178086,1.437368,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1993,-0.371801,-0.584936,0.675354,-0.228477,0.675192,1.294330,-1.143645,1.295326,0.265680,0.516246,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [117]:
X_test_cat = X_test.select_dtypes('object')

test_dums = ohe.transform(X_test_cat)

test_dums_df = pd.DataFrame(test_dums, columns=ohe.get_feature_names(), index=X_test_cat.index)

In [119]:
X_test_nums = X_test.select_dtypes(['int64','float64'])

test_nums = ss.transform(X_test_nums)

test_nums_df = pd.DataFrame(test_nums, columns=X_test_nums.columns, index=X_test_nums.index)

In [121]:
X_test_clean = pd.concat([test_nums_df, test_dums_df], axis=1)

X_test_clean.head()

,account length,number vmail messages,total day minutes,total day calls,total day charge,total eve minutes,total eve calls,total eve charge,total night minutes,total night calls,...,x0_VA,x0_VT,x0_WA,x0_WI,x0_WV,x0_WY,x1_415,x1_510,x2_yes,x3_yes
438,0.311486,-0.584936,-0.452712,-0.379362,-0.452767,2.562980,0.300651,2.562705,-0.219520,1.181501,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2674,-0.852632,-0.584936,-1.297381,0.827714,-1.297113,0.329524,1.197110,0.329704,-0.239243,2.102624,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1345,-0.068118,-0.584936,-3.305080,-5.056782,-3.305141,-0.810881,1.495930,-0.810008,-0.659356,-0.609571,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1957,1.171920,-0.584936,0.610946,-1.083490,0.611325,0.067112,-0.446399,0.067408,-0.874343,0.669766,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2148,-0.118732,-0.584936,-0.655138,0.073292,-0.655194,0.473554,-1.342858,0.473619,0.535893,-0.456051,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [122]:
knn = KNeighborsClassifier()

knn.fit(X_train_clean, y_train)

KNeighborsClassifier()

In [123]:
scores = cross_val_score(estimator=knn, X= X_train_clean, y= y_train, cv=10)
scores

array([0.87640449, 0.86516854, 0.88764045, 0.88389513, 0.88389513,
       0.89138577, 0.86842105, 0.87218045, 0.87969925, 0.90225564])

In [124]:
np.median(scores)

0.8817971896032215

In [125]:
knn.score(X_test_clean, y_test)

0.8935532233883059

In [133]:
knn3 = KNeighborsClassifier(n_neighbors=7)
knn3.fit(X_train_clean, y_train)

KNeighborsClassifier(n_neighbors=7)

In [134]:
scores = cross_val_score(estimator=knn3, X= X_train_clean, y= y_train, cv=10)
scores

array([0.87640449, 0.86891386, 0.88389513, 0.89138577, 0.89138577,
       0.89513109, 0.87218045, 0.86466165, 0.88345865, 0.89473684])

In [135]:
np.median(scores)

0.8836768888513419

In [136]:
knn3.score(X_test_clean, y_test)

0.8845577211394303